In [ ]:
using Pkg
Pkg.activate("FokkerPlanck")

In [ ]:
Pkg.add(["Distributions", "CairoMakie", "FHist"])

In [ ]:
using LinearAlgebra
using Distributions
using CairoMakie
using FHist

In [ ]:
# simulation duration
t_max = 200.0
# simulation step size
τ = 0.003
# size of the square on which we want to solve the PDE
BW = 15.0
# histogram discretization size
δ = 0.75

# we will compute the solution at two time steps T1 and T2
T1 = 5.0
T2 = 10.0


# output directory where the simulation frames will be stored
# create this folder before running simulation!
output_path = "fokker_planck/";

In [ ]:
function histogram_2d(x=[], y=[])
    Hist2D((x, y); binedges = (-BW:δ:BW,-BW:δ:BW)).bincounts
end

# a helper function for permuting particle dims for plotting
function permparticle(D)
    D2 = zeros(size(D))
    D2[:, 1] = D[:, 1]
    D2[:, 2] = D[:, 3]
    D2[:, 3] = D[:, 2]
    return D2
end

# a function for visualizing the particles
# this will save the image in `output_path` specified above
function plot_particles(step)
    global solution_T1, solution_T2, active_particles, δ
    fig = Figure(size=(1500, 800), fontsize=20.0)
    
    ax1 = Axis3(fig[1, 1], title = "A Monte Carlo Solution of Heat Equation", azimuth = -0.1 * pi, elevation = 0.1 * pi,
        aspect = (1, 3, 1), zlabel="y", xlabel="x", ylabel="Time", viewmode=:fitzoom)
    xlims!(-BW, BW)
    zlims!(-BW, BW)
    ylims!(0, T2)
    
    tail = 250
    
    heatmap!(ax1, -BW:δ:BW, -BW:δ:BW, solution_T2, transformation=(:xz, T2), colormap=:inferno)
    
    
    hidexdecorations!(ax1)
    hidezdecorations!(ax1)
    
    for ip in 1:length(active_particles)
        path = permparticle(stack(active_particles[ip])')[max((end-tail), 1):end, :]
        lines!(path, linewidth=0.75, color=(:gray, 1.0))
        scatter!(path[end, :]', markersize=7.0, color="#2d2d2d")
    end
    
    heatmap!(ax1, -BW:δ:BW, -BW:δ:BW, solution_T1, transformation=(:xz, T1), alpha=0.9, colormap=:inferno)
    
    fname = lpad(step, 4, "0")
    save("$(output_path)/$(fname).png", fig)
end

In [ ]:
# simulation time
t = 0

# array of particles
active_particles = []


# initialize empty histograms for solution at T1 and T2 steps
solution_T1 = histogram_2d()
solution_T2 = histogram_2d()


# In the simulation we are going to generate a series of brownian particles
# these particles will be generated based on a Poisson process with rate λ
# we also increase λ gradually during simulation, you can change this and use a constant λ
dynamic_λ(t, τ) = (12.0 - 11*t/t_max)*τ;


# the time of generating next particle
t_next_particle = rand(Exponential(dynamic_λ(t, τ)))

# here we will visualize simulation for every 75 steps
plot_period = 75
plot_step = 0
plotted_frames = 0

# main simulation loop
while t <= t_max
    # check if we should create a particle
    if t >= t_next_particle
        X = zeros(3)
        X[1:2] = 0.75*randn(2)
        X[3] = 0.0
        particle = [X]
        push!(active_particles, particle)
        t_next_particle = t + rand(Exponential(dynamic_λ(t, τ)))
    end

    particles_to_be_deleted = []
    
    # update position of particles
    for ip in 1:length(active_particles)
        X = zeros(3)
        ζ = randn(2)
        X[1:2] = active_particles[ip][end][1:2] + sqrt(2τ) * ζ
        X[3] = active_particles[ip][end][3] + τ
        push!(active_particles[ip], X)
   
        # update solution T1 if the particle has reached T1
        if length(active_particles[ip]) == T1÷τ
            solution_T1 += histogram_2d([X[1]], [X[2]])
        end

        # update solution T2 if the particle has reached T2 and delete the particle
        if X[3] >= T2
            solution_T2 += histogram_2d([X[1]], [X[2]])
            push!(particles_to_be_deleted, ip)
        end
    end
    
    deleteat!(active_particles, particles_to_be_deleted)

    if plot_step % plot_period == 1
        plotted_frames += 1
        plot_particles(plotted_frames)
    end
    plot_step += 1
    
    t += τ 
end